# Proyecto

Notas importantes

https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification?select=predictive_maintenance.csv

Revisar  los hiper parametros de regresion para regulizacion en xgboost de regresion, verificar y ajustar hiperparametros de clasificacion.

Ver como hacer un balanceo en el nuevo de clasificacion

poner una descripcion de cada variable

## Descripción del proyecto.


Participantes:


*   De Jesús Martinez Carlos Alberto
*   Miguel Esaú Rivera Román
*   Ivan Tonathiu López Rosales



## **Librerías Importadas**

In [ ]:
# Librerías para manejo de datos
import pandas as pd
import numpy as np

# Librerías para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para escalado y estandarización
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Librerías para división de datos (entrenamiento y prueba)
from sklearn.model_selection import train_test_split

# Librerías para modelos de aprendizaje automático - Regresión
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor

# Librerías para evaluación de modelos - Regresión
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Librerías para generación de datos sintéticos
from imblearn.over_sampling import SMOTE

# Librerías para modelos de aprendizaje automático - Clasificación
from sklearn.ensemble import RandomForestClassifier

# Librerías para evaluación de modelos - Clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Librerías para validación cruzada y optimización
from sklearn.model_selection import cross_val_score

## **Carga y Exploración del Dataset**

### *Carga del dataset*

In [ ]:
# Importamos nuestro dataset
data = pd.read_csv("predictive_maintenance.csv")

### Filas y Columnas

In [ ]:
data.head()

Descripción general:
Nuestro dataset contiene información sobre el funcionamiento de distintos productos con variables como temperatura, velocidad de rotación, torque y desgaste de la herramienta. Algunas columnas son categóricas y pueden ser transformadas, mientras que otras pueden ser eliminadas por no aportar valor analítico.

### Informacion del dataset

In [ ]:
data.info()

Datos completos y sin valores faltantes:
Al observar la estructura del dataset, podemos confirmar que no existen valores faltantes en ninguna de las columnas. Todas las entradas en las 10 columnas están completas (10000 valores no nulos por columna). Esto significa que no será necesario realizar ninguna imputación de valores ni eliminar filas o columnas debido a datos ausentes. Esto facilita el proceso de análisis, ya que no debemos preocuparnos por la limpieza de datos relacionada con la falta de información.

## **Análisis Exploratorio de Datos**

### *Detección de desbalanceo*

#### Failure Type

In [ ]:
data['Failure Type'].value_counts()

**Desbalanceo en la variable 'Failure Type':**

El dataset presenta un desbalance significativo entre las clases de Failure Type, con una gran mayoría de registros en la categoría No Failure. Este desbalance puede afectar el rendimiento del modelo, ya que los algoritmos tienden a favorecer las clases más representadas. Para mitigar este problema, es recomendable aplicar técnicas de balanceo como  SMOTE (Synthetic Minority Over-sampling Technique), que generan muestras sintéticas de las clases minoritarias.

In [ ]:
data['Failure Type'].value_counts().plot(kind='bar', color='orange', figsize=(8, 5))

# Etiquetas y título
plt.xlabel("Tipo de Falla")
plt.ylabel("Frecuencia")
plt.title("Frecuencia de Tipos de Fallas")
plt.xticks(rotation=25)

# Mostrar la gráfica
plt.show()

**Gráfica de Failure Type:**

La gráfica muestra claramente el desbalanceo entre las clases de Failure Type, evidenciando cómo No Failure domina en comparación con las demás categorías de fallas. Este desbalance puede tener un impacto negativo en el rendimiento del modelo, ya que los algoritmos podrían sesgarse hacia la clase más representada.

#### "Target"

In [ ]:
data['Target'].value_counts()

**Desbalanceo en el target 'Failure' vs 'No Failure':**

En la variable Target, que indica si una máquina falló (1) o no (0), también se observa un fuerte desbalanceo, con una abrumadora mayoría de casos en los que no hay falla. Este desbalance podría afectar negativamente el rendimiento del modelo, por lo que es importante aplicar técnicas de balanceo de clases para mejorar la predicción de la clase minoritaria.

In [ ]:
data['Target'].value_counts().plot(kind='bar', color='orange', figsize=(8, 5))

# Etiquetas y título
plt.xlabel("Categoría de Target")
plt.ylabel("Frecuencia")
plt.title("Frecuencia de Categorías en Target")
plt.xticks(rotation=25)

# Mostrar la gráfica
plt.show()

**Gráfica del Target:**

La gráfica visualiza claramente el desbalanceo en la variable Target, donde la clase No Failure (0) domina sobre Failure (1). Este desbalance resalta la necesidad de aplicar técnicas de balanceo para mejorar el rendimiento del modelo.

### *Distribución de variables numericas*

In [ ]:
data[['Air temperature [K]','Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]']].hist(bins=50, figsize=(20,15), color='orange')
plt.show()

# Mostrar el gráfico
plt.show()

**Gráfica de distribución de variables numéricas:**

La gráfica muestra la distribución de algunas de nuestras características numéricas más importantes, como Air temperature [K], Process temperature [K], Rotational speed [rpm] y Torque [Nm]. Estas variables podrían tener un gran impacto en la predicción de fallas en las máquinas, ya que reflejan aspectos clave del funcionamiento de los equipos.

### *Valores atipicos con BoxPlot*

**Identificación de valores atípicos con Boxplot:**

En esta sección, utilizamos boxplots para verificar la presencia de valores atípicos en nuestras variables numéricas. Los boxplots nos permiten visualizar la distribución de los datos y detectar posibles anomalías.

**Boxplots en color salmón:** Estos representan las variables que no contienen valores atípicos. La distribución de los datos se muestra dentro de los rangos esperados, sin ningún valor fuera de lo común.


---


**Boxplots en color rojo:** Estos muestran las variables que sí contienen valores atípicos. Los valores que se encuentran fuera de los límites del boxplot (los puntos que sobresalen) son considerados atípicos y podrían ser tratados dependiendo de su naturaleza.

#### Temperatura del aire

In [ ]:
# Crear un boxplot para visualizar valores atípicos en la temperatura del aire
plt.figure(figsize=(10, 6))
sns.boxplot(y=data['Air temperature [K]'], color='salmon')
plt.title('Diagrama de Caja: Temperatura del Aire')
plt.ylabel('Temperatura [K]')
plt.show()

**Sin valores atipicos**

#### Temperatura del proceso

In [ ]:
# Crear un boxplot para visualizar valores atípicos en la temperatura del proceso
plt.figure(figsize=(10, 6))
sns.boxplot(y=data['Process temperature [K]'], color='salmon')
plt.title('Diagrama de Caja: Temperatura del Proceso')
plt.ylabel('Temperatura [K]')
plt.show()

**Sin valores atipicos**

#### Velocidad rotacional

In [ ]:
# Crear un boxplot para visualizar valores atípicos en la velocidad rotacional
plt.figure(figsize=(10, 6))
sns.boxplot(y=data['Rotational speed [rpm]'], color="red")
plt.title('Diagrama de Caja: Velocidad Rotacional')
plt.ylabel('Velocidad [rpm]')
plt.show()

**Con valores atipicos**

In [ ]:
# Calcular cuartiles e IQR
Q1, Q3 = data['Rotational speed [rpm]'].quantile([0.25, 0.75])
IQR = Q3 - Q1

# Definir límites de valores atípicos
lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

# Identificar valores atípicos
outliers = (data['Rotational speed [rpm]'] < lim_inf) | (data['Rotational speed [rpm]'] > lim_sup+200)
num_outliers = outliers.sum()

# Reemplazar valores atípicos
data['Rotational speed [rpm]'] = data['Rotational speed [rpm]'].clip(lim_inf, lim_sup)
num_reemplazados = num_outliers  # Ya que cada outlier se reemplaza

# Mostrar resultados
print(f"Cantidad de valores atípicos detectados: {num_outliers}")
print(f"Cantidad de valores reemplazados: {num_reemplazados}")

#### Torque

In [ ]:
# Crear un boxplot para visualizar valores atípicos en el torque
plt.figure(figsize=(10, 6))
sns.boxplot(y=data['Torque [Nm]'], color='red')
plt.title('Diagrama de Caja: Torque')
plt.ylabel('Torque [Nm]')
plt.show()

*Con valores atipicos*

In [ ]:
# Calcular cuartiles e IQR
Q1, Q3 = data['Torque [Nm]'].quantile([0.25, 0.75])
IQR = Q3 - Q1

# Definir límites de valores atípicos
lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

# Identificar valores atípicos
outliers = (data['Torque [Nm]'] < lim_inf) | (data['Torque [Nm]'] > lim_sup)
num_outliers = outliers.sum()

# Reemplazar valores atípicos
data['Torque [Nm]'] = data['Torque [Nm]'].clip(lim_inf, lim_sup)
num_reemplazados = num_outliers  # Cada outlier es reemplazado

# Mostrar resultados
print(f"Cantidad de valores atípicos detectados: {num_outliers}")
print(f"Cantidad de valores reemplazados: {num_reemplazados}")

#### Tool Wear (Corregir)

In [ ]:
# Crear un boxplot para visualizar valores atípicos  tool weear (Corregir)
plt.figure(figsize=(10, 6))
sns.boxplot(y=data['Tool wear [min]'], color='salmon')
plt.title('Diagrama de Caja: Tool')
plt.ylabel('Tool')
plt.show()

*Sin valores atipicos*

## **Limpieza de datos**

### *Valores Nulos*

Verificamos que no haya valores nulos en nuestro dataset.

In [ ]:
data.isnull().sum()

Al no contar con valores nulos no es necesario hacer imputacion o eliminacion de columnas.

### *Eliminar columnar Irrelevantes o Innecesarias.*

Eliminamos columnas que no ocuparemos

In [ ]:
data = data.drop(columns=["UDI", "Product ID"])

# Mostramos como quedaria nuestro dataset
data.head()

## **Preprocesamiento de Datos.**

### Mapeo de Nuestro Targets

En esta seccion corta lo que estamos haciendo es mapear nuestros targets para que los modelos puedan leerlo con mas facilidad.

PD: Ocupamos mapeo manual en lugar de usar One-HotEncoding o LabelEncoding por que se nos hace mas comodo y en nuestro caso el One-Hot no, nos serviria en nuestro caso, en el caso del LabelEncoding decidimos hacer el mapeo Manual por comidad ya que no son muchas variables categorias a mapear.

In [ ]:
# Minutos de desgaste
data['Type'] = data['Type'].map({'L': 2, 'M': 3, 'H': 5})


# Aplicar Label Encoding
data['Failure Type'] = data['Failure Type'].map({
    "No Failure": 1,
    "Heat Dissipation Failure": 2,
    "Power Failure": 3,
    "Overstrain Failure": 4,
    "Tool Wear Failure": 5,
    "Random Failures": 6
})

In [ ]:
data.head()

## **Modelos de Regresión**

### *Análisis Exploratorio de Datos*

In [ ]:
# Calcular la matriz de correlación
correlacion_matrix = data.corr()

# Extraer la correlación de las features con 'Rotational speed [rpm]'
correlacion_target = correlacion_matrix["Rotational speed [rpm]"].sort_values(ascending=False)

# Mostrar las correlaciones
correlacion_target


In [ ]:
#Grafica de la matriz de correlacion
plt.figure(figsize=(10, 6))
sns.heatmap(correlacion_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.show()

#### HeatMap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(correlacion_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.show()


Podemos ver que hay algunas correlaciones muy bajas y algunas correlaciones negativas la cual ocuparemos para modificar las features que ocuparemos.

Torque [Nm]: -0.911734 (correlación negativa fuerte)

Air temperature [K]: 0.021305 (correlación muy baja)

Process temperature [K]: 0.019892 (correlación muy baja)

### *Preparación y Preprocesamiento de Datos*

#### Seleccion y división de features

In [ ]:
# Variables independientes (características)
X_regresion = data[['Torque [Nm]', 'Air temperature [K]', 'Process temperature [K]']]

# Variable dependiente (objetivo)
y_regresion = data['Rotational speed [rpm]']

#### Escalado de Datos


Escalamos los datos para que nuestros modelos puedan leerlo de una manera mas sencilla.

En nuestro caso utilizaremos conforme nos de mejor precision nuestro modelos entre MinMaxScaler y StandarScaler

In [ ]:
# Creamos una instancia para escalar con Min-Max
scaler_min_max_r = MinMaxScaler()
# Creamos una instancia para escalar con Min-Max
scaler_standar_r = StandardScaler() # Este es el que ocuparemos

# Ajustar y transformar los datos
X_escalado_min_max_r = scaler_min_max_r.fit_transform(X_regresion)  # X es el conjunto de datos y esta escalado con MinMaxScaler()
X_escalado_standar_r = scaler_standar_r.fit_transform(X_regresion)  # X es el conjunto de datos y esta escalado con StandardScaler()

#### Division de data

Insertamos nuestros datos escalados y luego dividimos dichos datos conforme vayamos probando la precision.

In [ ]:
# Dividir los datos en entrenamiento (70%) y prueba (30%)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_escalado_standar_r, y_regresion, test_size=0.3, random_state=42)

### *Instancia y creacion del modelo*

In [ ]:
# Modelo de regresion con RandomForestRegressor

model_rf_r = RandomForestRegressor(
    n_estimators=400,      # Aumentamos el número de árboles para mejorar la estabilidad
    max_depth=5,           # Reducimos la profundidad para evitar overfitting
    min_samples_split=20,  # Requiere más muestras para hacer una división
    min_samples_leaf=10,   # Cada hoja debe tener al menos 10 muestras
    max_features=0.7,      # Reducimos el número de características usadas por árbol
    random_state=42
)

# Modelo de regresion con XGBRegressor

model_xgb_r = XGBRegressor(
    n_estimators=500,   # Mantiene el número de árboles
    learning_rate=0.05,  # Velocidad de aprendizaje moderada
    max_depth=1,        # Aumenta ligeramente la profundidad para captar más patrones
    reg_lambda=10,       # Regularización L2 (reduce pesos grandes)
    reg_alpha=5,      # Regularización L1 (elimina pesos innecesarios)
    subsample=0.8,      # Usa el 80% de los datos por árbol
    colsample_bytree=0.8, # Usa el 80% de las características en cada árbol
    random_state=42
)

# Modelo de regresion con GradientBoostingRegressor

model_gb_r = GradientBoostingRegressor(
    n_estimators=400,              # Incrementa el número de árboles
    max_depth=5,                    # Mantén la profundidad moderada
    learning_rate=0.01,             # Reduce la tasa de aprendizaje
    random_state=42,                # Semilla aleatoria para reproducibilidad
    min_samples_split=20,           # Aumenta el número mínimo de muestras para dividir
    min_samples_leaf=10,            # Aumenta el número mínimo de muestras en cada hoja
    subsample=0.8,                  # Usa el 80% de los datos para cada árbol
    max_features="sqrt"             # Usa la raíz cuadrada de las características
)

In [ ]:
"""
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import numpy as np

# Función objetivo para Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 20),  # Regularización L2
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
    }
    model = XGBRegressor(**params, random_state=42)
    return -np.mean(cross_val_score(model, X_train_r, y_train_r, cv=5, scoring='r2'))

# Optimización
study = optuna.create_study(direction='minimize')  # Minimizamos el negativo del R²
study.optimize(objective, n_trials=50)

# Mejores hiperparámetros
print("Mejores parámetros:", study.best_params)
"""

En este caso tambien agregamos otros hiperparametros como regulizacion, tiempo de entrenamiento, etc.
Para que almenos ligeramente

### *Entrenamiento del modelo*

Entrenamos nuestro modelo con los datos divididos en nuetros caso un 70% de entrenamiento tanto de X y Y

In [ ]:
# Entrenamos el modelo con RandomForestRegressor
model_rf_r.fit(X_train_r, y_train_r)

In [ ]:
# Entrenamos el modelo con XGBRegressor
model_xgb_r.fit(X_train_r, y_train_r)

In [ ]:
# Entrenamos el modelo con GradientBoostingRegressor
model_gb_r.fit(X_train_r, y_train_r)

### *Predicciones*

RandomForestRegressor

In [ ]:
# Predicción Test con Random Forest
y_test_pred_rf_r = model_rf_r.predict(X_test_r)
# Predicción Entrenamiento
y_train_pred_rf_r = model_rf_r.predict(X_train_r)

XGBRegressor

In [ ]:
# Predicción Test con XGBRegresor
y_test_pred_xgb_r = model_xgb_r.predict(X_test_r)
# Predicción Entrenamiento
y_train_pred_xgb_r = model_xgb_r.predict(X_train_r)

GradientBoostingRegressor

In [ ]:
# Predicción Test con GradientBoostingRegressor
y_test_pred_gb_r = model_gb_r.predict(X_test_r)
# Predicción Entrenamiento
y_train_pred_gb_r = model_gb_r.predict(X_train_r)

### *Graficas*

En esta grafica veremos que tanto se separa nuestros valores de nuestro modelo. Viendo que tan preciso fue.

#### *Colores (Ignorar)*

In [ ]:
colors1 = plt.cm.Blues(np.linspace(0.3, 1, 3))  # Colores de tonos azules
colors2 = plt.cm.Reds(np.linspace(0.3, 1, 3))   # Colores de tonos rojos
colors3 = plt.cm.Greens(np.linspace(0.3, 1, 3))  # Colores de tonos verdes

#### *RandomForestRegressor*

In [ ]:
# Graficar las predicciones vs los valores reales de RandomForestRegressor
plt.figure(figsize=(8,6))
plt.scatter(y_test_r, y_test_pred_rf_r, color=colors1[1])
plt.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], '--k', lw=2)
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (RandomForestRegressor)')
plt.show()

#### *XGBRegressor*

In [ ]:
# Graficar las predicciones vs los valores reales de XGBRegressor
plt.figure(figsize=(8,6))
plt.scatter(y_test_r, y_test_pred_xgb_r, color=colors2[1])
plt.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], '--k', lw=2)
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (XGBRegressor)')
plt.show()

#### *GradientBoostingRegressor*

In [ ]:
# Graficar las predicciones vs los valores reales de GradientBoostingRegressor
plt.figure(figsize=(8,6))
plt.scatter(y_test_r, y_test_pred_gb_r, color=colors3[1])
plt.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], '--k', lw=2)
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (GradientBoostingRegressor)')
plt.show()

### *Metricas*

Ahora utilizaremos metricas como MAE, MSE, RMSE y R2 para saber que tanto se equivoca nuestro modelo, si es que tenemos errores grandes y la "precision" de nuestro modelo.

#### *RandomForestRegressor*

In [ ]:
# Cálculo de métricas para entrenamiento
mae_train_xgb_r = mean_absolute_error(y_train_r, y_train_pred_xgb_r)
mse_train_xgb_r = mean_squared_error(y_train_r, y_train_pred_xgb_r)
rmse_train_xgb_r = np.sqrt(mse_train_xgb_r)
r2_train_xgb_r = r2_score(y_train_r, y_train_pred_xgb_r)

# Cálculo de métricas para prueba
mae_test_xgb_r = mean_absolute_error(y_test_r, y_test_pred_xgb_r)
mse_test_xgb_r = mean_squared_error(y_test_r, y_test_pred_xgb_r)
rmse_test_xgb_r = np.sqrt(mse_test_xgb_r)
r2_test_xgb_r = r2_score(y_test_r, y_test_pred_xgb_r)

# Imprimir el reporte
print("Reporte de Evaluación del Modelo de Regresión:")
print("Entrenamiento:")
print(f"  MAE: {mae_train_xgb_r:.4f}")
print(f"  MSE: {mse_train_xgb_r:.4f}")
print(f"  RMSE: {rmse_train_xgb_r:.4f}")
print(f"  R^2 Score: {r2_train_xgb_r:.4f}")

print("\nPrueba:")
print(f"  MAE: {mae_test_xgb_r:.4f}")
print(f"  MSE: {mse_test_xgb_r:.4f}")
print(f"  RMSE: {rmse_test_xgb_r:.4f}")
print(f"  R^2 Score: {r2_test_xgb_r:.4f}")

#### *XGBRegressor*

In [ ]:
# Cálculo de métricas para entrenamiento
mae_train_rf_r = mean_absolute_error(y_train_r, y_train_pred_rf_r)
mse_train_rf_r = mean_squared_error(y_train_r, y_train_pred_rf_r)
rmse_train_rf_r = np.sqrt(mse_train_rf_r)
r2_train_rf_r = r2_score(y_train_r, y_train_pred_rf_r)

# Cálculo de métricas para prueba
mae_test_rf_r = mean_absolute_error(y_test_r, y_test_pred_rf_r)
mse_test_rf_r = mean_squared_error(y_test_r, y_test_pred_rf_r)
rmse_test_rf_r = np.sqrt(mse_test_rf_r)
r2_test_rf_r = r2_score(y_test_r, y_test_pred_rf_r)

# Imprimir el reporte
print("Reporte de Evaluación del Modelo de Regresión:")
print("Entrenamiento:")
print(f"  MAE: {mae_train_rf_r:.4f}")
print(f"  MSE: {mse_train_rf_r:.4f}")
print(f"  RMSE: {rmse_train_rf_r:.4f}")
print(f"  R^2 Score: {r2_train_rf_r:.4f}")

print("\nPrueba:")
print(f"  MAE: {mae_test_rf_r:.4f}")
print(f"  MSE: {mse_test_rf_r:.4f}")
print(f"  RMSE: {rmse_test_rf_r:.4f}")
print(f"  R^2 Score: {r2_test_rf_r:.4f}")

#### *GradientBoostingRegressor*

In [ ]:
# Cálculo de métricas para entrenamiento
mae_train_gbm_r = mean_absolute_error(y_train_r, y_train_pred_gb_r)
mse_train_gbm_r = mean_squared_error(y_train_r, y_train_pred_gb_r)
rmse_train_gbm_r = np.sqrt(mse_train_gbm_r)
r2_train_gbm_r = r2_score(y_train_r, y_train_pred_gb_r)

# Cálculo de métricas para prueba
mae_test_gbm_r = mean_absolute_error(y_test_r, y_test_pred_gb_r)
mse_test_gbm_r = mean_squared_error(y_test_r, y_test_pred_gb_r)
rmse_test_gbm_r = np.sqrt(mse_test_gbm_r)
r2_test_gbm_r = r2_score(y_test_r, y_test_pred_gb_r)

# Imprimir el reporte
print("Reporte de Evaluación del Modelo de Regresión:")
print("Entrenamiento:")
print(f"  MAE: {mae_train_gbm_r:.4f}")
print(f"  MSE: {mse_train_gbm_r:.4f}")
print(f"  RMSE: {rmse_train_gbm_r:.4f}")
print(f"  R^2 Score: {r2_train_gbm_r:.4f}")

print("\nPrueba:")
print(f"  MAE: {mae_test_gbm_r:.4f}")
print(f"  MSE: {mse_test_gbm_r:.4f}")
print(f"  RMSE: {rmse_test_gbm_r:.4f}")
print(f"  R^2 Score: {r2_test_gbm_r:.4f}")

### *Votacion Suave*

In [ ]:
from sklearn.ensemble import VotingRegressor

# Crear el modelo de votación suave
model_votacion_suave_r = VotingRegressor(estimators=[
    ('rf', model_rf_r),
    ('xgb', model_xgb_r),
    ('gbr', model_gb_r)
])

# Entrenamiento
model_votacion_suave_r.fit(X_train_r, y_train_r)

# Predicción en el conjunto de entrenamiento y prueba
predicciones_train_vs_r = model_votacion_suave_r.predict(X_train_r)
predicciones_test_vs_r = model_votacion_suave_r.predict(X_test_r)

# Calcular las métricas para el conjunto de entrenamiento
mae_train_vs_r = mean_absolute_error(y_train_r, predicciones_train_vs_r)
mse_train_vs_r = mean_squared_error(y_train_r, predicciones_train_vs_r)
rmse_train_vs_r = np.sqrt(mse_train_vs_r)
r2_train_vs_r = r2_score(y_train_r, predicciones_train_vs_r)

# Calcular las métricas para el conjunto de prueba
mae_test_vs_r = mean_absolute_error(y_test_r, predicciones_test_vs_r)
mse_test_vs_r = mean_squared_error(y_test_r, predicciones_test_vs_r)
rmse_test_vs_r = np.sqrt(mse_test_vs_r)
r2_test_vs_r = r2_score(y_test_r, predicciones_test_vs_r)

# Imprimir el reporte
print("Reporte de Evaluación del Modelo de Votación Suave:")

# Métricas de Entrenamiento
print("\nEntrenamiento:")
print(f"  MAE: {mae_train_vs_r:.4f}")
print(f"  MSE: {mse_train_vs_r:.4f}")
print(f"  RMSE: {rmse_train_vs_r:.4f}")
print(f"  R^2 Score: {r2_train_vs_r:.4f}")

# Métricas de Prueba
print("\nPrueba:")
print(f"  MAE: {mae_test_vs_r:.4f}")
print(f"  MSE: {mse_test_vs_r:.4f}")
print(f"  RMSE: {rmse_test_vs_r:.4f}")
print(f"  R^2 Score: {r2_test_vs_r:.4f}")

## **Modelo Clasificación**

### Seleccion y división de features

In [ ]:
# Separamos nuestro target de nuestras features para hacerle smote por separado.
x_clasificacion_uno_rf = data.drop(columns=['Failure Type', 'Target'])
y_clasificacion_uno_rf = data['Target']  # Variable objetivo | No Failure: 0 / Failure:1

### Smote

In [ ]:
# Aplicar SMOTE
smote_uno_rf = SMOTE(sampling_strategy='auto', random_state=42)
x_resampled_uno_rf, y_resampled_uno_rf = smote_uno_rf.fit_resample(x_clasificacion_uno_rf, y_clasificacion_uno_rf)

# Convertir de nuevo a DataFrame
data_resampled_uno_rf = pd.DataFrame(x_resampled_uno_rf, columns=x_clasificacion_uno_rf.columns)
data_resampled_uno_rf['Target'] = y_resampled_uno_rf

# Verificar el balance después de aplicar SMOTE
print(data_resampled_uno_rf['Target'].value_counts())

# Variables a ocupar para el smote
x_clasificacion_smote_uno_rf = data_resampled_uno_rf.drop(columns=['Target'])
y_clasificacion_smote_uno_rf = data_resampled_uno_rf['Target']

### **Division de data**

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba en 70% de entrenamiento y 30% prueba
X_train_c_uno, X_test_c_uno, y_train_c_uno, y_test_c_uno = train_test_split(
    x_clasificacion_smote_uno_rf, y_clasificacion_smote_uno_rf, test_size=0.3, random_state=42
)

### *Instancia y creación del modelo*

In [ ]:
# Crear el modelo Random Forest utilizando "Target como Objetivo"
modelo_rf_c_uno = RandomForestClassifier(
    n_estimators=75,   # Reduce el número de árboles
    max_depth=10,      # Limita la profundidad de los árboles
    min_samples_split=5,  # Evita divisiones con muy pocos datos
    min_samples_leaf=3,   # Cada hoja debe tener al menos 3 muestras
    max_features='sqrt',  # Reduce las características usadas en cada división
    random_state=42
)

### *Entrenamiento del modelo*

In [ ]:
# Entrenar el modelo
modelo_rf_c_uno.fit(X_train_c_uno, y_train_c_uno)

### *Predicciones*

In [ ]:
# Predecir con el modelo en entrenamiento y prueba
y_train_pred_c_uno = modelo_rf_c_uno.predict(X_train_c_uno)
y_test_pred_c_uno = modelo_rf_c_uno.predict(X_test_c_uno)

### *Metricas*

In [ ]:
# Evaluar el modelo en entrenamiento
accuracy_train_c_uno = accuracy_score(y_train_c_uno, y_train_pred_c_uno)
print("Entrenamiento:")
print(f"  Accuracy: {accuracy_train_c_uno:.4f}")
print(classification_report(y_train_c_uno, y_train_pred_c_uno))

# Evaluar el modelo en prueba
accuracy_test_c_uno = accuracy_score(y_test_c_uno, y_test_pred_c_uno)
print("\nPrueba:")
print(f"  Accuracy: {accuracy_test_c_uno:.4f}")
print(classification_report(y_test_c_uno, y_test_pred_c_uno))

CSV

In [ ]:
accuracy_train_c_uno = accuracy_score(y_train_c_uno, y_train_pred_c_uno)
report_train = classification_report(y_train_c_uno, y_train_pred_c_uno, output_dict=True)

# Evaluar el modelo en prueba
accuracy_test_c_uno = accuracy_score(y_test_c_uno, y_test_pred_c_uno)
report_test = classification_report(y_test_c_uno, y_test_pred_c_uno, output_dict=True)

# Convertir los reportes en DataFrames
df_train = pd.DataFrame(report_train).transpose()
df_train['Dataset'] = 'Entrenamiento'

df_test = pd.DataFrame(report_test).transpose()
df_test['Dataset'] = 'Prueba'

# Unir los DataFrames
df_results = pd.concat([df_train, df_test])

# Guardar a CSV
df_results.to_csv("resultados_modelo.csv", index=True)
print("Resultados guardados en 'resultados_modelo.csv'")

### Grafica

In [ ]:
# Diccionario de nombres de clases para este modelo
clase_nombres_uno = {
    0: "No Failure",
    1: "Failure"
}

# Obtener las clases únicas
clases_uno = np.unique(y_test_c_uno)

# Crear subgráficos para entrenamiento y prueba
fig, axes = plt.subplots(len(clases_uno), 2, figsize=(12, len(clases_uno) * 4))

for i, clase in enumerate(clases_uno):
    # Matriz de confusión para la clase actual
    cm_train_uno = confusion_matrix(y_train_c_uno == clase, y_train_pred_c_uno == clase)
    cm_test_uno = confusion_matrix(y_test_c_uno == clase, y_test_pred_c_uno == clase)

    # Obtener el nombre de la clase actual
    nombre_clase_uno = clase_nombres_uno.get(clase, f"Clase {clase}")

    # Gráfico de entrenamiento
    sns.heatmap(cm_train_uno, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'], ax=axes[i, 0])
    axes[i, 0].set_title(f'Matriz de Confusión - {nombre_clase_uno} (Entrenamiento)')
    axes[i, 0].set_xlabel('Predicho')
    axes[i, 0].set_ylabel('Real')

    # Gráfico de prueba
    sns.heatmap(cm_test_uno, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'], ax=axes[i, 1])
    axes[i, 1].set_title(f'Matriz de Confusión - {nombre_clase_uno} (Prueba)')
    axes[i, 1].set_xlabel('Predicho')
    axes[i, 1].set_ylabel('Real')

plt.tight_layout()
plt.show()

### Curva de aprendizaje

In [ ]:
from sklearn.model_selection import learning_curve

# Curva de aprendizaje
def plot_learning_curve(model, x_clasificacion_smote, y_clasificacion_smote):
    train_sizes, train_scores, test_scores = learning_curve(model, x_clasificacion_smote, y_clasificacion_smote, cv=5, scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_mean, 'o-', color="r", label="Entrenamiento")
    plt.plot(train_sizes, test_mean, 'o-', color="g", label="Prueba")
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color="g")

    plt.xlabel("Tamaño del conjunto de entrenamiento")
    plt.ylabel("Precisión")
    plt.title("Curva de Aprendizaje")
    plt.legend()
    plt.show()

# Generar la curva de aprendizaje
plot_learning_curve(modelo_rf_c_uno, x_clasificacion_smote_uno_rf, y_clasificacion_smote_uno_rf)

## **Clasificacion - 2**

### Seleccion y división de features

In [ ]:
# Separamos nuestro target de nuestras features para hacerle smote por separado.
x_clasificacion_dos_rf = data.drop(columns=['Failure Type', 'Target'])
y_clasificacion_dos_rf = data['Failure Type']  # Variable objetivo

# Donde las siguientes variables se mapearon quedando de tal forma.
"""
    "No Failure": 1,
    "Heat Dissipation Failure": 2,
    "Power Failure": 3,
    "Overstrain Failure": 4,
    "Tool Wear Failure": 5,
    "Random Failures": 6

"""

### Smote

In [ ]:
# Aplicar SMOTE
smote_dos_rf = SMOTE(sampling_strategy='auto', random_state=42)
x_resampled_dos_rf, y_resampled_dos_rf = smote_dos_rf.fit_resample(x_clasificacion_dos_rf, y_clasificacion_dos_rf)

# Convertir de nuevo a DataFrame
data_resampled_dos_rf = pd.DataFrame(x_resampled_dos_rf, columns=x_clasificacion_dos_rf.columns)
data_resampled_dos_rf['Target'] = y_resampled_dos_rf

# Verificar el balance después de aplicar SMOTE
print(data_resampled_dos_rf['Target'].value_counts())


x_clasificacion_smote_dos_rf = data_resampled_dos_rf.drop(columns=['Target'])
y_clasificacion_smote_dos_rf = data_resampled_dos_rf['Target']

### **Division de data**

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train_c_dos, X_test_c_dos, y_train_c_dos, y_test_c_dos = train_test_split(
    x_clasificacion_smote_dos_rf, y_clasificacion_smote_dos_rf, test_size=0.3, random_state=42
)

### *Instancia y creación del modelo*

In [ ]:
# Crear el modelo Random Forest
modelo_rf_c_dos = RandomForestClassifier(
    n_estimators=75,   # Reduce el número de árboles
    max_depth=10,      # Limita la profundidad de los árboles
    min_samples_split=5,  # Evita divisiones con muy pocos datos
    min_samples_leaf=3,   # Cada hoja debe tener al menos 3 muestras
    max_features='sqrt',  # Reduce las características usadas en cada división
    random_state=42
)

### *Entrenamiento del modelo*

In [ ]:
# Entrenar el modelo
modelo_rf_c_dos.fit(X_train_c_dos, y_train_c_dos)


### *Predicciones*

In [ ]:
# Predecir con el modelo en entrenamiento y prueba
y_train_pred_c_rf_dos = modelo_rf_c_dos.predict(X_train_c_dos)
y_test_pred_c_rf_dos = modelo_rf_c_dos.predict(X_test_c_dos)

### *Metricas*

In [ ]:
# Evaluar el modelo en entrenamiento
accuracy_train_c_dos = accuracy_score(y_train_c_dos, y_train_pred_c_rf_dos)
print("Entrenamiento:")
print(f"  Accuracy: {accuracy_train_c_dos:.4f}")
print(classification_report(y_train_c_dos, y_train_pred_c_rf_dos))

# Evaluar el modelo en prueba
accuracy_test_c_dos = accuracy_score(y_test_c_dos, y_test_pred_c_rf_dos)
print("\nPrueba:")
print(f"  Accuracy: {accuracy_test_c_dos:.4f}")
print(classification_report(y_test_c_dos, y_test_pred_c_rf_dos))


### Grafica

In [ ]:
# Diccionario de clases con sus nombres específicos para este modelo
clase_nombres_dos = {
    1: "No Failure",
    2: "Heat Dissipation Failure",
    3: "Power Failure",
    4: "Overstrain Failure",
    5: "Tool Wear Failure",
    6: "Random Failures"
}

# Obtener las clases únicas
clases_dos = np.unique(y_test_c_dos)

# Crear subgráficos para entrenamiento y prueba
fig, axes = plt.subplots(len(clases_dos), 2, figsize=(12, len(clases_dos) * 4))

for i, clase in enumerate(clases_dos):
    # Matriz de confusión para la clase actual
    cm_train_dos = confusion_matrix(y_train_c_dos == clase, y_train_pred_c_rf_dos == clase)
    cm_test_dos = confusion_matrix(y_test_c_dos == clase, y_test_pred_c_rf_dos == clase)

    # Obtener el nombre de la clase actual
    nombre_clase_dos = clase_nombres_dos.get(clase, f"Clase {clase}")

    # Gráfico de entrenamiento
    sns.heatmap(cm_train_dos, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'], ax=axes[i, 0])
    axes[i, 0].set_title(f'Matriz de Confusión - {nombre_clase_dos} (Entrenamiento)')
    axes[i, 0].set_xlabel('Predicho')
    axes[i, 0].set_ylabel('Real')

    # Gráfico de prueba
    sns.heatmap(cm_test_dos, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'], ax=axes[i, 1])
    axes[i, 1].set_title(f'Matriz de Confusión - {nombre_clase_dos} (Prueba)')
    axes[i, 1].set_xlabel('Predicho')
    axes[i, 1].set_ylabel('Real')

plt.tight_layout()
plt.show()


### Curva de Aprendizaje

In [ ]:
from sklearn.model_selection import learning_curve

# Curva de aprendizaje
def plot_learning_curve(model, x_clasificacion_smote, y_clasificacion_smote):
    train_sizes, train_scores, test_scores = learning_curve(model, x_clasificacion_smote, y_clasificacion_smote, cv=5, scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_mean, 'o-', color="r", label="Entrenamiento")
    plt.plot(train_sizes, test_mean, 'o-', color="g", label="Prueba")
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color="g")

    plt.xlabel("Tamaño del conjunto de entrenamiento")
    plt.ylabel("Precisión")
    plt.title("Curva de Aprendizaje")
    plt.legend()
    plt.show()

# Generar la curva de aprendizaje
plot_learning_curve(modelo_rf_c_dos, x_clasificacion_smote_dos_rf, y_clasificacion_smote_dos_rf)

## **Modelo Clustering**

In [ ]:
# Seleccionar características numéricas relevantes
features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
X = data[features]

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA
# Reducir la dimensionalidad con PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Mostrar las cargas (componentes principales)
pca_components = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2', 'PC3'])
print("Cargas de las Componentes Principales:")
print(pca_components)

# Mostrar la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_
print("\nVarianza Explicada por Componente:")
print(explained_variance)

In [ ]:
data['Failure Type'].value_counts()

    "No Failure": 1
    "Heat Dissipation Failure": 2
    "Power Failure": 3
    "Overstrain Failure": 4
    "Tool Wear Failure": 5
    "Random Failures": 6

PCA Sirve para desglosar o redimensionar el data para que sea mas pequeño, reducir el ruido y sacar una correlacion

*   PC1 separa los datos en función de la velocidad de rotación y el torque.
*   PC2 refleja principalmente diferencias en las temperaturas.
*   PC3 está casi completamente relacionada con el desgaste de la herramienta.

In [ ]:
# Aplicar K-Means con 6 clusters
from sklearn.cluster import KMeans # Import the KMeans class from sklearn.cluster
kmeans = KMeans(n_clusters=6, random_state=42)
labels_kmeans = kmeans.fit_predict(X_scaled)

# Visualizar los resultados usando PCA
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels_kmeans, palette='tab10', s=50)
plt.title('Clustering con K-Means y PCA (2D)')
plt.xlabel('PC-1')
plt.ylabel('PC-2')
plt.legend(title='Cluster')
plt.show()

# Mostrar la distribución de puntos por cluster
cluster_distribution = pd.Series(labels_kmeans).value_counts().sort_index()
cluster_distribution

*   PC1 separa los datos en función de la velocidad de rotación y el torque.
*   PC2 refleja principalmente diferencias en las temperaturas.
*   PC3 está casi completamente relacionada con el desgaste de la herramienta.

In [ ]:
import plotly.graph_objects as go

# Convertir a DataFrame para facilitar la manipulación
df = pd.DataFrame(X_pca, columns=['Componente Principal 1', 'Componente Principal 2', 'Componente Principal 3'])
df['Cluster'] = labels_kmeans

palette = sns.color_palette("tab10", n_colors=6)
colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' for r, g, b in palette]

# Crear la figura 3D
fig = go.Figure()

# Agregar puntos para cada cluster
for cluster in range(6):
    cluster_data = df[df['Cluster'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=cluster_data['Componente Principal 1'],
        y=cluster_data['Componente Principal 2'],
        z=cluster_data['Componente Principal 3'],
        mode='markers',
        marker=dict(size=4, color=colors[cluster]),
        name=f'Cluster {cluster}'
    ))

# Configurar diseño
fig.update_layout(
    title='Clustering con K-Means y PCA (3D)',
    scene=dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'
    ),
    legend_title='Cluster'
)

# Mostrar la figura interactiva
fig.show()


*   PC1 separa los datos en función de la velocidad de rotación y el torque.
*   PC2 refleja principalmente diferencias en las temperaturas.
*   PC3 está casi completamente relacionada con el desgaste de la herramienta.